# Downloading images from Wikimedia Commons

This notebook defines a few functions to download images with specified search terms from Wikimedia, via the mediawiki search API.

The results are used in the [Parallel face detection notebook](Parallel%20face%20detection.ipynb).

For some caching, you can use requests_cache:

    pip install requests_cache

In [ ]:
import sys, os

import requests
try:
    import requests_cache
except ImportError:
    print("no cache, no worries")
else:
    requests_cache.install_cache("mediawiki")

api_url = "http://commons.wikimedia.org/w/api.php"

In [ ]:
def api_request(**kwargs):
    """Make a request of the Wikimedia Commons API
    
    Returns data after parsing JSON
    """
    sys.stdout.write('.')
    sys.stdout.flush()
    

    params = dict(
        action='query',
        format='json',
    )
    params.update(kwargs)
    r = requests.get(api_url, params=params)
    r.raise_for_status()
    return r.json()


import json

def search_images(search, limit=100, size_limit=2e6):
    """search wikimedia commons for a given term
    
    returns a list of `limit` URLs for images
    """
    continue_params = {}
    while limit > 0:
        data = api_request(
                srnamespace=6,
                prop='imageinfo',
                list='search',
                srsearch=search,
                srlimit=min(limit, 50),
                **continue_params
        )
#         continue_params = data['query-continue']['search']
        continue_params = data['continue']

        total = data['query']['searchinfo']['totalhits']
        results = data['query']['search']
        for r in results:
            title = r['title']
            
            data = api_request(
                            prop='imageinfo',
                            titles=title,
                            iiprop='url|size|mime')
            imageinfo = list(data['query']['pages'].values())[0]['imageinfo'][0]
            if imageinfo['mime'] in ('image/png', 'image/jpeg') and imageinfo['size'] <= size_limit:
                yield imageinfo['url']
                limit -= 1
                if limit <= 0:
                    return
#             else:
#                 import pprint
#                 pprint.pprint(imageinfo)


In [ ]:
def download_images(search, n):
    """download images from mediawiki commons to folders based on the search term"""
    if not os.path.exists('images'):
        os.mkdir('images')
    tagdir = os.path.join('images', search)
    if not os.path.exists(tagdir):
        os.mkdir(tagdir)
    for url in search_images(search, n):
        r = requests.get(url)
        fname = url.rsplit('/')[-1]
        dest = os.path.join(tagdir, fname)
        # print("downloading %s => %s" % (url, dest))
        sys.stdout.write('+')
        sys.stdout.flush()
        with open(dest, 'wb') as f:
            f.write(r.content)

In [ ]:
download_images('castle', 100)

In [ ]:
download_images('portrait', 100)

In [ ]:
download_images('face', 100)

In [ ]:
download_images('child', 100)